In [3]:
import pandas as pd

data = pd.read_csv("textual_data_an1_cleaned_final.csv")

print(data.head())

                                   clean_tweet_2  label
0                                          nigga      4
1                                horses retarded      5
2  nigga momma youngboy spitting real shit nigga      0
3            rt xxsugvngxx ran holy nigga today       1
4                       everybody calling nigger      1


In [4]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import nltk

# Verify if 'punkt' is available
nltk.download('punkt', download_dir='/Users/admin/nltk_data')

sentences = data['clean_tweet_2'].dropna().tolist()
#tokenized_sentences = [sentence.split() for sentence in sentences]
tokenizer = RegexpTokenizer(r'\w+')
tokenized_sentences = [tokenizer.tokenize(str(sentence).lower()) for sentence in sentences]

#model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, sg=1, epochs=10)

#model.save("word2vec_model.model")

[nltk_data] Downloading package punkt to /Users/admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
model = Word2Vec.load('word2vec_model.model')

word_vector = model.wv['shit']
print("Vector for 'shit':", word_vector)

similar_words = model.wv.most_similar('shit')
print("Words similar to 'shit':", similar_words)

Vector for 'shit': [-1.9590057e-02  4.6684235e-01  7.6496118e-01  7.1472056e-02
  7.4470842e-01 -5.0849015e-01  7.2395462e-01  6.5145570e-01
 -9.7402000e-01 -9.4522440e-01  3.0409154e-01 -8.9401555e-01
 -8.6123921e-02 -1.3225059e-01 -2.1727771e-01  1.5976471e-01
  5.5095989e-01 -3.9442542e-01 -9.4595104e-02 -4.1664425e-01
  6.2709384e-02  2.2566906e-01  4.3575704e-02 -1.8051888e-01
  1.4805755e-01  2.4423726e-01 -3.7822351e-01 -2.5321850e-01
 -5.6976473e-01 -5.5646533e-01 -5.4964959e-03 -1.9575089e-01
 -2.2708194e-01 -8.8444851e-02 -4.9538112e-01  1.2278016e-01
  2.0446534e-01 -5.7703382e-03 -1.6811025e-01 -5.7831174e-01
  2.0288713e-01  6.7173398e-01  2.1752685e-01  2.2173375e-01
  6.0447073e-01 -2.6183934e-03 -3.8476551e-01 -1.7862551e-01
  3.2777292e-01  5.9678084e-01  3.7988600e-01 -1.4047587e-01
 -6.2954897e-01 -1.8691503e-01 -1.0080524e-01  2.6467201e-01
  1.2537839e-01 -4.7455542e-04 -3.8506609e-01  6.9666165e-01
 -5.4413533e-01 -4.0494233e-01  1.5303372e-01 -6.2520456e-01
 -6.9

In [6]:

# Check most similar words
word = "hate"
similar_words = model.wv.most_similar(word, topn=5)
print(f"Words similar to '{word}': {similar_words}")


Words similar to 'hate': [('hates', 0.7087909579277039), ('istg', 0.702265739440918), ('fault', 0.6987146139144897), ('kidnap', 0.692625105381012), ('supportive', 0.6921353340148926)]


In [7]:
from gensim.models import FastText

#model = FastText(vector_size=100, window=5, min_count=1, sg=1, epochs=10)
#model.build_vocab(tokenized_sentences)
#model.train(tokenized_sentences, total_examples=len(tokenized_sentences), epochs=10)

#model.save("fasttext_model.model")

In [8]:
model = FastText.load("fasttext_model.model")

word_vector = model.wv['hate']
print("Vector for 'hate':", word_vector)

similar_words = model.wv.most_similar('hate')
print("Similar words:", similar_words)

Vector for 'hate': [-0.19896089 -0.03855652  0.21756397  0.4290057   0.35268864  0.68681043
 -0.7992246   0.20737763  0.99332976 -0.63186306 -0.12635213 -0.04417552
  0.05359266  0.30471575  0.05943685 -0.5588839   0.7302943  -0.15063119
 -0.50517595 -0.38255358 -0.0675873   0.6960804  -0.40953735 -0.12459321
 -0.34085524 -0.3950263  -0.06828337  0.22408985  0.12599343 -0.04847601
  0.9342074   0.7227501   0.24040486  0.42846844 -0.08402305  0.5176666
  0.5890232   0.8959477  -0.3460574   0.12036419 -0.4729769  -0.23858634
  0.6336515  -0.5059421  -0.8992771  -0.0083879  -0.7330792   0.4467388
  0.51504976  0.27611798 -0.20217358 -0.50223315  0.20131956  0.27600825
 -0.00985072 -0.65852374 -0.13819632 -0.09978468  0.40393248  0.07709184
  0.2858255  -0.40327787 -0.29199857  0.85438746 -0.17418267  0.8320237
 -0.03298299 -0.14447047 -0.18150316  0.5372792   0.18773109  0.3217547
  0.29163474  0.01376706  0.3485192  -0.02856916  0.37934053 -0.35196486
  0.02059063  0.7738005  -0.3987366 

In [9]:
import numpy as np
from gensim.models import FastText
from scipy.sparse import csr_matrix
from collections import defaultdict

# Load FastText model
model = FastText.load("fasttext_model.model")

# Sample sentences
sentences = data['clean_tweet_2']
# Ensure all sentences are strings and handle NaN values
sentences = [str(sentence) for sentence in sentences if pd.notnull(sentence)]

# Create vocabulary
vocab = list(model.wv.index_to_key)
# Create a word-to-index dictionary for faster lookups
word_to_index = defaultdict(lambda: -1, {word: i for i, word in enumerate(vocab)})

# Create embedding matrix (X) - word embeddings
embedding_dim = model.vector_size
X = np.zeros((len(vocab), embedding_dim), dtype=np.float32)
print("Shape of X:", X.shape)
for i, word in enumerate(vocab):
    X[i] = model.wv[word]

# Create hypergraph incidence matrix (H)
num_sentences = len(sentences)
H = np.zeros((len(vocab), num_sentences), dtype=np.float32)

print("Shape of H:", H.shape)

for j, sentence in enumerate(sentences):
    for word in sentence.split():
        i = word_to_index[word]
        if i != -1:
            H[i, j] = 1

# Convert to sparse matrices
X_sparse = csr_matrix(X)  # Feature matrix
H_sparse = csr_matrix(H)  # Hypergraph incidence matrix

print("Feature Matrix Shape (X):", X_sparse.shape)
print("Incidence Matrix Shape (H):", H_sparse.shape)


Shape of X: (60373, 100)
Shape of H: (60373, 149822)
Feature Matrix Shape (X): (60373, 100)
Incidence Matrix Shape (H): (60373, 149822)
